In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
from PIL import Image
from keras.engine import  Model
import os
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate, MaxPooling2D,Flatten
import pickle
import glob
from keras.models import *
from keras.layers import *
import keras.backend as K
from keras.layers.core import Lambda, Dense
import math
from scipy import spatial
from skimage import color
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# !pip install flask gevent requests pillow

     |████████████████████████████████| 5.6MB 5.4MB/s 
     |████████████████████████████████| 163kB 28.0MB/s 
     |████████████████████████████████| 245kB 37.2MB/s 


In [ ]:
from keras.applications import ResNet50
from keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils
from PIL import Image
import numpy as np
import flask
import io
app = flask.Flask(__name__)
model=None

In [ ]:
def load_model():
    global model
    base_model = keras.applications.ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=None,
        pooling='avg'
    )
    x=tf.keras.layers.Dropout(.5)(base_model.output)
    x=Dense(512)(x)
    x=tf.keras.layers.Dropout(.5)(x)
    x=Dense(256)(x)
    x=tf.keras.layers.Dropout(.5)(x)
    x=Dense(64)(x)
    x=tf.keras.layers.Dropout(.5)(x)
    predictions = Dense(10, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.load_weights('/content/drive/My Drive/Colab Notebooks/Kaggle/dog-breed-identification/mdl_wts1.hdf5')

In [ ]:
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
def prepare_image(image, target):
    if image.mode != "RGB":
        image = image.convert("RGB")
    image = image.resize(target)
    image = img_to_array(image)
    # image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    image=np.reshape(image,(1,224,224,3))
    return image

In [ ]:
# !pip install flask_ngrok

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask
import base64
app = Flask(__name__)
run_with_ngrok(app)
@app.route("/predict", methods=["POST"])
def predict():
    data = {}
    if flask.request.method == "POST":
        image = Image.open(io.BytesIO(base64.b64decode(flask.request.form['b64'])))
        image = prepare_image(image, target=(224, 224))
        preds = model.predict(image)
        tg={0: 'beagle',
            1: 'chihuahua',
            2: 'doberman',
            3: 'french_bulldog',
            4: 'golden_retriever',
            5: 'malamute',
            6: 'pug',
            7: 'saint_bernard',
            8: 'scottish_deerhound',
            9: 'tibetan_mastiff'}
        data["breed"] = tg[np.argmax(preds[0])]
        data["score"]=float(preds[0][np.argmax(preds[0])])
    return flask.jsonify(data)
if __name__ == "__main__":
    load_model()
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://1dbc96acc648.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [13/Mar/2021 10:03:37] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2021 10:04:02] "POST /predict HTTP/1.1" 200 -
